In [20]:
import httpx
import polars as pl

In [21]:
INE_BASE_URL = "https://servicios.ine.es/wstempus/js/ES"

client = httpx.Client(
    base_url=INE_BASE_URL,
    limits=httpx.Limits(max_keepalive_connections=20),
    transport=httpx.HTTPTransport(retries=3),
)

In [22]:
def ine_request(client: httpx.Client, endpoint, paginate=True):
    page = 1
    data = []

    while True:
        params = {"det": 10}

        if paginate:
            params["page"] = page

        response = client.get(
            f"/{endpoint}", params=params, follow_redirects=True
        ).json()

        if not response:
            break

        data.extend(response)

        if len(response) < 500 or not paginate:
            break

        page += 1

    return data

In [23]:
# https://servicios.ine.es/wstempus/js/ES/OPERACIONES_DISPONIBLES

operaciones_disponibles = pl.DataFrame(ine_request(client, "OPERACIONES_DISPONIBLES"))
print(operaciones_disponibles.shape)
operaciones_disponibles.sample(5)

(108, 6)


Id,Cod_IOE,Nombre,Codigo,Referencia,Url
i64,str,str,str,list[struct[3]],str
205,"""30310""","""Flujos de la Población Activa""","""EFPA""","[{139,""Estadística de flujos de la población activa"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176907&idp=1254735976595""}]",null
213,"""30466""","""Estadística de Condenados: Adu…","""CONA""","[{169,""Estadística de condenados: Adultos"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176793&idp=1254735573206""}]",null
303,"""30187""","""Encuesta Trimestral de Coste L…","""ETCL""","[{84,""Encuesta trimestral de coste laboral"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736045053&idp=1254735976596""}]",null
249,"""30063""","""Encuesta Coyuntural sobre Stoc…","""ECSE""","[{31,""Encuesta coyuntural sobre stocks y existencias"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176995&idp=1254735576799""}]",null
234,"""30209""","""Estadística de Movilidad Labor…","""EMLG""","[{93,""Estadística de movilidad laboral y geográfica"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176909&idp=1254735976597""}]",null


In [24]:
# https://servicios.ine.es/wstempus/js/ES/VARIABLES?det=10

variables = pl.DataFrame(ine_request(client, "VARIABLES"))
variables.sample(5)

Id,Nombre,Codigo
i64,str,str
729,"""Lugar de nacimiento del cónyug…",""""""
177,"""Personas entre 16 y 25 años qu…",""""""
277,"""Grupos 1992""",""""""
745,"""Duración del procedimiento""",""""""
476,"""SI/NO""",""""""


In [25]:
t = []

for row in operaciones_disponibles.rows(named=True):
    tablas_operacion_url = f"TABLAS_OPERACION/{row["Id"]}"
    tablas_operacion = ine_request(client, tablas_operacion_url)

    t.extend(tablas_operacion)

In [26]:
import os

os.makedirs("dataset", exist_ok=True)

tablas = pl.json_normalize(t)
print(tablas.shape)
tablas.write_ndjson("dataset/tablas.jsonl")
tablas.sample(5)

(4934, 50)


Id,Nombre,Codigo,Anyo_Periodo_ini,FechaRef_fin,Ultima_Modificacion,Periodicidad.Id,Periodicidad.Nombre,Periodicidad.Codigo,Publicacion.Id,Publicacion.Nombre,Publicacion.Periodicidad.Id,Publicacion.Periodicidad.Nombre,Publicacion.Periodicidad.Codigo,Publicacion.Operacion,Publicacion.PubFechaAct.Id,Publicacion.PubFechaAct.Nombre,Publicacion.PubFechaAct.Fecha,Publicacion.PubFechaAct.Periodo.Id,Publicacion.PubFechaAct.Periodo.Valor,Publicacion.PubFechaAct.Periodo.Periodicidad.Id,Publicacion.PubFechaAct.Periodo.Periodicidad.Nombre,Publicacion.PubFechaAct.Periodo.Periodicidad.Codigo,Publicacion.PubFechaAct.Periodo.Dia_inicio,Publicacion.PubFechaAct.Periodo.Mes_inicio,Publicacion.PubFechaAct.Periodo.Codigo,Publicacion.PubFechaAct.Periodo.Nombre,Publicacion.PubFechaAct.Periodo.Nombre_largo,Publicacion.PubFechaAct.Anyo,Periodo_ini.Id,Periodo_ini.Valor,Periodo_ini.Periodicidad.Id,Periodo_ini.Periodicidad.Nombre,Periodo_ini.Periodicidad.Codigo,Periodo_ini.Dia_inicio,Periodo_ini.Mes_inicio,Periodo_ini.Codigo,Periodo_ini.Nombre,Periodo_ini.Nombre_largo,Anyo_Periodo_fin,Periodo_fin.Id,Periodo_fin.Valor,Periodo_fin.Periodicidad.Id,Periodo_fin.Periodicidad.Nombre,Periodo_fin.Periodicidad.Codigo,Periodo_fin.Dia_inicio,Periodo_fin.Mes_inicio,Periodo_fin.Codigo,Periodo_fin.Nombre,Periodo_fin.Nombre_largo
i64,str,str,str,str,i64,i64,str,str,i64,str,i64,str,str,list[struct[5]],i64,str,i64,i64,i64,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str
49237,"""Por sectores de actividad y co…","""CCAA""","""2008""","""null""",1721811600000,12,"""Anual""","""A""",162,"""Encuesta Anual de Coste Labora…",12,"""Anual""","""A""","[{139,""30188"",""Encuesta Anual de Coste Laboral"",""EACL"",[{85,""Encuesta anual de coste laboral"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736060920&idp=1254735976596""}]}]",10625,"""Encuesta Anual de Coste Labora…",1721811600000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2023,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",null,null,null,null,null,null,null,null,null,null,null
30932,"""Porcentaje de población con in…","""MUN-DIST-SECC""","""2015""","""null""",1698660000000,12,"""Anual""","""A""",507,"""Atlas de Distribución de Renta…",12,"""Anual""","""A""","[{353,""30325"",""Atlas de Distribución de Renta de los Hogares"",""ADRH"",null}]",10737,"""Atlas de Distribución de Renta…",1705572000000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2021,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",null,null,null,null,null,null,null,null,null,null,null
30692,"""Edad Media al Matrimonio por m…","""MUN""","""2014""",null,1700647200000,12,"""Anual""","""A""",38,"""Indicadores Demográficos Básic…",12,"""Anual""","""A""","[{33,""30264"",""Indicadores Demográficos Básicos"",""IDB"",[{126,""Indicadores demográficos básicos"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177003&idp=1254735573002""}]}]",10892,"""Indicadores Demográficos Básic…",1721811600000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2022,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""","""2021""",28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año"""
1991,"""Índice de precios de campings …","""2002_NAC""","""2003""","""null""",1725001200000,1,"""Mensual""","""M""",2,"""Encuesta de ocupación en aloja…",1,"""Mensual""","""M""","[{61,""30222"",""Índice de Precios de Apartamentos Turísticos"",""IPAP"",[{99,""Apartamentos turísticos: encuesta de ocupación e índice de precios"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176962&idp=1254735576863""}]}, {62,""30223"",""Índice de Precios de Camping"",""IPAC"",[{100,""Campings: encuesta de ocupación e índice de precios"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176961&idp=1254735576863""}]}, … {328,""30239"",""Encuesta de Ocupación en Alb

In [27]:
def generate_readme_content(data):
    return f"""# {data['Nombre']}

- **ID:** {data['Id']}
- **Código:** {data['Codigo']}
- **Año de Inicio del Periodo:** {data['Anyo_Periodo_ini']}
- **Fecha de Referencia Final:** {data['FechaRef_fin']}
- **Última Modificación:** {data['Ultima_Modificacion']}
- **Periodicidad:** {data['Periodicidad.Nombre']} ({data['Periodicidad.Codigo']})

## Publicación

- **ID:** {data['Publicacion.Id']}
- **Nombre:** {data['Publicacion.Nombre']}
- **Periodicidad:** {data['Publicacion.Periodicidad.Nombre']} ({data['Publicacion.Periodicidad.Codigo']})

### Operación

- **ID:** {data['Publicacion.Operacion'][0]['Id']}
- **Código IOE:** {data['Publicacion.Operacion'][0]['Cod_IOE']}
- **Nombre:** {data['Publicacion.Operacion'][0]['Nombre']}
- **Referencia:** {data['Publicacion.Operacion'][0]['Referencia']}

### Última Publicación

- **ID:** {data['Publicacion.PubFechaAct.Id']}
- **Nombre:** {data['Publicacion.PubFechaAct.Nombre']}
- **Fecha:** {data['Publicacion.PubFechaAct.Fecha']}
- **Periodo ID:** {data['Publicacion.PubFechaAct.Periodo.Id']}
- **Periodo Valor:** {data['Publicacion.PubFechaAct.Periodo.Valor']}
- **Periodo Periodicidad:** {data['Publicacion.PubFechaAct.Periodo.Periodicidad.Nombre']} ({data['Publicacion.PubFechaAct.Periodo.Periodicidad.Codigo']})
- **Periodo Día de Inicio:** {data['Publicacion.PubFechaAct.Periodo.Dia_inicio']}
- **Periodo Mes de Inicio:** {data['Publicacion.PubFechaAct.Periodo.Mes_inicio']}
- **Periodo Código:** {data['Publicacion.PubFechaAct.Periodo.Codigo']}
- **Periodo Nombre:** {data['Publicacion.PubFechaAct.Periodo.Nombre']}
- **Periodo Nombre Largo:** {data['Publicacion.PubFechaAct.Periodo.Nombre_largo']}
- **Año:** {data['Publicacion.PubFechaAct.Anyo']}

## Periodo Inicial

- **ID:** {data['Periodo_ini.Id']}
- **Valor:** {data['Periodo_ini.Valor']}
- **Periodicidad:** {data['Periodo_ini.Periodicidad.Nombre']} ({data['Periodo_ini.Periodicidad.Codigo']})
- **Día de Inicio:** {data['Periodo_ini.Dia_inicio']}
- **Mes de Inicio:** {data['Periodo_ini.Mes_inicio']}
- **Código:** {data['Periodo_ini.Codigo']}
- **Nombre:** {data['Periodo_ini.Nombre']}
- **Nombre Largo:** {data['Periodo_ini.Nombre_largo']}

## Periodo Final

- **Año de Fin del Periodo:** {data['Anyo_Periodo_fin']}
- **ID:** {data['Periodo_fin.Id']}
- **Valor:** {data['Periodo_fin.Valor']}
- **Periodicidad:** {data['Periodo_fin.Periodicidad.Nombre']} ({data['Periodo_fin.Periodicidad.Codigo']})
- **Día de Inicio:** {data['Periodo_fin.Dia_inicio']}
- **Mes de Inicio:** {data['Periodo_fin.Mes_inicio']}
- **Código:** {data['Periodo_fin.Codigo']}
- **Nombre:** {data['Periodo_fin.Nombre']}
- **Nombre Largo:** {data['Periodo_fin.Nombre_largo']}

## JSON

```json
{data}
```
"""

In [28]:
def get_series_tabla_url(tabla_id):
    return (
        f"https://servicios.ine.es/wstempus/jsCache/ES/SERIES_TABLA/{tabla_id}?det=10"
    )


def get_tablas_download_url(tabla_id):
    return f"https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/{tabla_id}.csv"


tablas = tablas.with_columns(
    pl.col("Id")
    .map_elements(get_series_tabla_url, return_dtype=pl.String)
    .alias("series_tabla_url"),
    pl.col("Id")
    .map_elements(get_tablas_download_url, return_dtype=pl.String)
    .alias("tablas_download_url"),
)

In [29]:
tablas = tablas.sample(10)

In [30]:
with open("series.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['series_tabla_url']}\n")
        f.write("\tout=metadata.json\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

with open("tablas.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['tablas_download_url']}\n")
        f.write(f"\tout={t['Id']}.csv\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

In [31]:
import subprocess

series_result = subprocess.run(
    [
        "aria2c",
        "-i",
        "series.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
    ],
    capture_output=True,
    text=True,
)

In [32]:
print(series_result.stdout)

[DL:784KiB][#738f3b 0B/0B][#e9f6c8 0B/0B][#ebae53 0B/0B][#2df43b 0B/0B][#23a4b0 0B/0B](+2)
[DL:448KiB][#738f3b 0B/0B][#e9f6c8 0B/0B][#ebae53 0B/0B][#2df43b 0B/0B][#23a4b0 0B/0B](+2)
[DL:393KiB][#738f3b 55KiB/0B][#e9f6c8 44KiB/0B][#ebae53 53KiB/0B][#2df43b 25KiB/0B][#23a4b0 41KiB/0B](+2)
[DL:3.4MiB][#738f3b 2.5MiB/0B][#ebae53 2.6MiB/0B][#2df43b 2.5MiB/0B][#b593a0 2.5MiB/0B][#c994d4 2.4MiB/0B]
[DL:5.2MiB][#738f3b 5.7MiB/0B][#2df43b 5.7MiB/0B][#b593a0 5.8MiB/0B][#c994d4 5.5MiB/0B]
[DL:6.3MiB][#738f3b 8.7MiB/0B][#2df43b 8.7MiB/0B][#b593a0 8.8MiB/0B][#c994d4 8.5MiB/0B]
[DL:7.0MiB][#738f3b 11MiB/0B][#2df43b 11MiB/0B][#b593a0 11MiB/0B][#c994d4 11MiB/0B]
[DL:7.4MiB][#738f3b 14MiB/0B][#2df43b 14MiB/0B][#c994d4 14MiB/0B]
[DL:7.7MiB][#738f3b 18MiB/0B][#2df43b 18MiB/0B][#c994d4 18MiB/0B]
[DL:11MiB][#738f3b 21MiB/0B][#c994d4 21MiB/0B]
[#c994d4 24MiB/0B CN:1 DL:3.0MiB]
[#c994d4 27MiB/0B CN:1 DL:3.0MiB]
[#c994d4 30MiB/0B CN:1 DL:3.1MiB]
[#c994d4 33MiB/0B CN:1 DL:3.1MiB]
[#c994d4 37MiB/0B CN:1 DL:3.2M

In [33]:
tablas_result = subprocess.run(
    [
        "aria2c",
        "-i",
        "tablas.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
    ],
    capture_output=True,
    text=True,
)

In [34]:
print(tablas_result.stdout)

[DL:25MiB][#f5a222 3.6MiB/0B][#d5fedd 4.0MiB/0B][#508d1f 4.0MiB/0B][#3fbdaa 4.4MiB/0B][#9370b5 3.4MiB/0B]
[DL:26MiB][#f5a222 9.5MiB/0B][#d5fedd 9.8MiB/0B][#3fbdaa 10MiB/0B][#9370b5 9.1MiB/0B]
[DL:24MiB][#f5a222 15MiB/0B][#9370b5 15MiB/0B]
[#9370b5 21MiB/0B CN:1 DL:5.9MiB]
[#9370b5 28MiB/0B CN:1 DL:6.1MiB]
[#9370b5 35MiB/0B CN:1 DL:6.2MiB]
[#9370b5 42MiB/0B CN:1 DL:6.3MiB]
[#9370b5 48MiB/0B CN:1 DL:6.3MiB]
[#9370b5 55MiB/0B CN:1 DL:6.3MiB]

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
fb2b34|OK  |    21KiB/s|dataset/tablas/20264/20264.csv
c2986b|OK  |   370KiB/s|dataset/tablas/66202/66202.csv
e50167|OK  |   893KiB/s|dataset/tablas/65719/65719.csv
e5dd70|OK  |   798KiB/s|dataset/tablas/58700/58700.csv
03ce7a|OK  |   1.0MiB/s|dataset/tablas/70173/70173.csv
508d1f|OK  |   4.8MiB/s|dataset/tablas/60111/60111.csv
3fbdaa|OK  |   5.3MiB/s|dataset/tablas/6532/6532.csv
d5fedd|OK  |   5.3MiB/s|dataset/tablas/310

In [35]:
import glob

from tqdm import tqdm

csv_files = glob.glob("dataset/tablas/*/*.csv")

for file in tqdm(csv_files):
    filename = file.split(".")[-2].split("/")[-1]

    table_metadata = tablas.filter(pl.col("Id") == int(filename)).to_struct()[0]

    with open(f"dataset/tablas/{filename}/README.md", "w") as f:
        f.write(generate_readme_content(table_metadata))

    (
        pl.scan_csv(
            file,
            separator=";",
            ignore_errors=True,
            truncate_ragged_lines=True,
        ).sink_parquet(
            f"dataset/tablas/{filename}/datos.parquet",
            compression="zstd",
            row_group_size=1024**2,
            type_coercion=True,
        )
    )


100%|██████████| 10/10 [00:00<00:00, 41.80it/s]


In [36]:
import os

for file in csv_files:
    os.remove(file)

In [37]:
from huggingface_hub import HfApi

In [38]:
import os

api = HfApi(token=os.getenv("HUGGINGFACE_TOKEN"))

In [39]:
import shutil

shutil.copy("DATASET_README.md", "dataset/README.md")

api.upload_large_folder(
    folder_path="dataset", repo_id="davidgasquez/ine-sandbox", repo_type="dataset"
)

Recovering from metadata files: 100%|██████████| 32/32 [00:00<00:00, 9682.42it/s]





---------- 2024-09-30 12:12:51 (0:00:00) ----------
Files:   hashed 12/32 (1.5M/142.5M) | pre-uploaded: 0/0 (0.0/142.5M) (+32 unsure) | committed: 0/32 (0.0/142.5M) | ignored: 0
Workers: hashing: 13 | get upload mode: 1 | pre-uploading: 0 | committing: 0 | waiting: 0
---------------------------------------------------


metadata.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]





metadata.json:   3%|▎         | 344k/12.8M [00:00<00:04, 3.09MB/s]


















metadata.json:  14%|█▎        | 1.75M/12.8M [00:00<00:01, 7.78MB/s]


datos.parquet: 100%|██████████| 36.5k/36.5k [00:00<00:00, 121kB/s]


datos.parquet: 100%|██████████| 19.5k/19.5k [00:00<00:00, 64.0kB/s]
datos.parquet: 100%|██████████| 42.6k/42.6k [00:00<00:00, 127kB/s]



datos.parquet: 100%|██████████| 265k/265k [00:00<00:00, 519kB/s] 
datos.parquet: 100%|██████████| 38.5k/38.5k [00:00<00:00, 74.1kB/s]
datos.parquet: 100%|██████████| 52.7k/52.7k [00:00<00:00, 150kB/s]]



metadata.json:  54%|█████▍    | 6.95M/12.8M [00:00<00:00, 11.9MB/s]


datos.parquet: 100%|██████████| 15.0k/15.0k [00:00<00:00, 52.5kB/s]




metadata.json:  88%|████████▊ | 11.3M/12.8M [00:00<00:00, 13.4MB/s]









metadata.json:  99%|█████████▉| 12.7M/12.8M [00:01<00:00, 12.8MB/s]









datos.parquet: 100%|██████████| 104k/104k [00:00<00:00, 131kB/s]
datos.pa